In [1]:
data_var = '2023-12-10'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
3619,2023-12-10,Espanha Acb,08:30,Joventut,Obradoiro CAB,1.37,3.12,164.5,1.83,1.97,-7.5,2.03,1.75,lGw99H36,0.729927,0.320513,0.546448,0.507614,0.050440,0.0,0.0,NaN,NaN,196.442,63.199482,0.321721,256.066,131.314089,0.512813,0.000,0.0,187.11,427.80,0.0,0.0,0.0,0.0,0.551197,0.052103,0.104757,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
3620,2023-12-10,Espanha Acb,08:30,Real Madrid,Gran Canaria,1.19,4.85,169.5,1.80,1.96,-12.5,2.06,1.69,APv5Acl0,0.840336,0.206186,0.555556,0.510204,0.046522,0.0,0.0,NaN,NaN,100.184,9.758313,0.097404,165.306,72.902806,0.441017,0.000,0.0,97.37,191.90,0.0,0.0,0.0,0.0,0.856957,0.060179,0.139536,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
3621,2023-12-10,Espanha Acb,13:00,Baskonia,Tenerife,1.33,3.38,164.5,1.87,1.92,-7.5,2.03,1.75,lCmchJT7,0.751880,0.295858,0.534759,0.520833,0.047738,0.0,0.0,NaN,NaN,125.326,18.155957,0.144870,180.302,118.067727,0.654833,0.000,0.0,120.32,225.60,0.0,0.0,0.0,0.0,0.615528,0.018657,0.104757,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
3622,2023-12-10,Espanha Acb,13:00,Murcia,Valencia,1.55,2.47,158.5,1.88,1.90,-4.5,2.00,1.79,xStuH268,0.645161,0.404858,0.531915,0.526316,0.050020,0.0,0.0,NaN,NaN,139.830,48.565261,0.347316,125.888,36.054597,0.286402,0.000,0.0,131.25,138.37,0.0,0.0,0.0,0.0,0.323651,0.007483,0.078360,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
3623,2023-12-10,Espanha Acb,14:30,Zaragoza,Barcelona,4.26,1.23,162.5,1.83,1.95,7.5,2.01,1.78,GIuqGMLE,0.234742,0.813008,0.546448,0.512821,0.047750,0.0,0.0,NaN,NaN,196.768,48.366942,0.245807,105.178,16.006815,0.152188,0.000,0.0,222.22,108.04,0.0,0.0,0.0,0.0,0.780522,0.044896,0.085823,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3826,2023-12-10,Eua Ncaa,21:00,Temple,Albany,1.38,3.17,152.5,1.80,1.95,-8.5,2.10,1.67,dQAc5yIT,0.724638,0.315457,0.555556,0.512821,0.040095,0.8,0.0,NaN,NaN,152.350,39.585482,0.259833,144.408,70.765553,0.490039,133.862,0.0,174.85,148.78,0.0,0.0,0.0,0.0,0.556361,0.056569,0.161303,-2.42,-0.484,-0.785124,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
3827,2023-12-10,Eua Ncaa,22:00,Washington State,Grambling St.,1.01,20.00,137.5,1.80,1.95,-23.5,2.05,1.71,dOeIsxA4,0.990099,0.050000,0.555556,0.512821,0.040099,1.0,0.0,NaN,NaN,112.534,52.805821,0.469243,NaN,NaN,NaN,112.534,0.0,94.60,621.00,0.0,1.0,0.0,0.0,1.278244,0.056569,0.127881,-0.75,-0.150,-0.066667,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
3828,2023-12-10,Grécia Liga De Elite,10:00,Lefkadas,Milon Aons,2.06,1.69,147.5,1.82,1.88,1.5,1.90,1.80,OK4wrWBO,0.485437,0.591716,0.549451,0.531915,0.077153,0.0,0.0,NaN,NaN,200.626,113.876321,0.567605,112.962,22.170654,0.196266,0.000,0.0,80.24,127.82,0.0,0.0,0.0,0.0,0.139536,0.022933,0.038222,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
3829,2023-12-10,Israel Superliga,15:30,Ironi Kiryat Ata,Maccabi Ironi Ramat Gan,1.66,2.17,166.5,1.86,1.88,-4.5,2.10,1.62,pd6JU2HN,0.602410,0.460829,0.537634,0.531915,0.063239,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,139.00,222.40,0.0,0.0,0.0,0.0,0.188316,0.007563,0.182479,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
